# Collect Job Data with Generative AI

This notebook demonstrates how to collect job posts from [USAJOBS](https://developer.usajobs.gov/). 

Please note:
- If you find a data source that provides direct data download, downloading data is the easiest way.
- Otherwise, APIs can be used with the assistance of AI to collect data.
- Please avoid web crawling with AI, and always check the [robots.txt](https://developers.google.com/search/docs/crawling-indexing/robots/intro) file before crawling a website.

## Set up a Database and Request API Keys

Create a [MongoDB](www.mongodb.com) cluster and store the connection string in a safe place, such as AWS Secrets Manager. 
- key name: `api_key`
- key value: <`the connection string`>, you need to type the password
- secret name: `mongodb`

Request a [USAJOBS API key](https://developer.usajobs.gov/apirequest/) and store the key in a safe place, such as AWS Secrets Manager. 
- key name: `api_key`
- key value: <`the API key you received in email`>
- secret name: `usajobs`

You also need to store your email in AWS Secrets Manager:
- key name: `address`
- key value: <`the email you used in applying the API key`>
- secret name: `email`

## Install Python Packages

- jupyter-ai: the JupyterLab extension to call Generative AI models
- langchain-openai: the LangChain package to interact with OpenAI
- pymongo: manage the MongoDB database

In [10]:
pip install jupyter-ai~=1.0 # Because I am using JupyterLab V3, I need to use Jupyter-ai V1.0

  Using cached aiosqlite-0.20.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached deepmerge-2.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached jupyter_ai_magics-2.26.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached jupyterlab-3.6.8-py3-none-any.whl.metadata (12 kB)
  Using cached jupyter_ydoc-0.2.5-py3-none-any.whl.metadata (2.2 kB)
  Using cached jupyter_server_ydoc-0.8.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached nbclassic-1.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached notebook-6.5.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached jsonpath-ng-1.7.0.tar.gz (37 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached langchain_community-0.2.17-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
  Using cached jupyter_server_fileid-0.9.3-py3-none-any.whl.metadata (4.7 kB)
  Using cached ypy_websocket-0.8.4-py3-none-any

In [1]:
pip install jupyter-ai[all] # execute this cell if the AI model not in the ai list

  Using cached aiosqlite-0.20.0-py3-none-any.whl.metadata (4.3 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/setuptools/_distutils/dist.py:261: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      running egg_info
      creating /tmp/pip-pip-egg-info-3_5rvecm/sgmllib3k.egg-info
      writing /tmp/pip-pip-egg-info-3_5rvecm/sgmllib3k.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-3_5rvecm/sgmllib3k.egg-info/dependency_links.txt
      writing top-level names to /tmp/pip-pip-egg-info-3_5rvecm/sgmllib3k.egg-info/top_level.tx

In [2]:
pip install langchain-openai # skip this if you pip install jupyter-ai[all]

  Using cached langchain_openai-0.2.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 90.4 MB/s eta 0:00:00
Using cached packaging-24.1-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.0.2 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 72.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Secrets Manager Function

In [4]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [6]:
import pymongo
from pymongo import MongoClient
import json
import re
import os

os.environ["OPENAI_API_KEY"] = get_secret('openai')['api_key']
email = get_secret('email')['address']
mongodb_connect = get_secret('mongodb')['connection_string']
usa_jobs_key = get_secret('usajobs')['api_key']

## Connect to the MongoDB cluster

In [7]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
job_collection = db.job_collection #use or create a collection named job_collection


## Load the Jupyter AI Magic Commands

In [11]:
%load_ext jupyter_ai_magics

Check the available AI models, this is optional. 

In [12]:
%ai list

| Provider | Environment variable | Set? | Models |
|----------|----------------------|------|--------|
| `ai21` | `AI21_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | <ul><li>`ai21:j1-large`</li><li>`ai21:j1-grande`</li><li>`ai21:j1-jumbo`</li><li>`ai21:j1-grande-instruct`</li><li>`ai21:j2-large`</li><li>`ai21:j2-grande`</li><li>`ai21:j2-jumbo`</li><li>`ai21:j2-grande-instruct`</li><li>`ai21:j2-jumbo-instruct`</li></ul> |
| `gpt4all` | Not applicable. | <abbr title="Not applicable">N/A</abbr> | <ul><li>`gpt4all:ggml-gpt4all-j-v1.2-jazzy`</li><li>`gpt4all:ggml-gpt4all-j-v1.3-groovy`</li><li>`gpt4all:ggml-gpt4all-l13b-snoozy`</li><li>`gpt4all:mistral-7b-openorca.Q4_0`</li><li>`gpt4all:mistral-7b-instruct-v0.1.Q4_0`</li><li>`gpt4all:gpt4all-falcon-q4_0`</li><li>`gpt4all:wizardlm-13b-v1.2.Q4_0`</li><li>`gpt4all:nous-hermes-llama2-13b.Q4_0`</li><li>`gpt4all:gpt4all-13b-snoozy-q4_0`</li><li>`gpt4all:mpt-7b-chat-merges-q4_0`</li><li>`gpt4all:orca-mini-3b-gguf2-q4_0`</li><li>`gpt4all:starcoder-q4_0`</li><li>`gpt4all:rift-coder-v0-7b-q4_0`</li><li>`gpt4all:em_german_mistral_v01.Q4_0`</li></ul> |
| `huggingface_hub` | `HUGGINGFACEHUB_API_TOKEN` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | See [https://huggingface.co/models](https://huggingface.co/models) for a list of models. Pass a model's repository ID as the model ID; for example, `huggingface_hub:ExampleOwner/example-model`. |
| `qianfan` | `QIANFAN_AK`, `QIANFAN_SK` | <abbr title="You have not set all of these environment variables, so you cannot use this provider's models.">❌</abbr> | <ul><li>`qianfan:ERNIE-Bot`</li><li>`qianfan:ERNIE-Bot-4`</li></ul> |
| `togetherai` | `TOGETHER_API_KEY` | <abbr title="You have not set this environment variable, so you cannot use this provider's models.">❌</abbr> | <ul><li>`togetherai:Austism/chronos-hermes-13b`</li><li>`togetherai:DiscoResearch/DiscoLM-mixtral-8x7b-v2`</li><li>`togetherai:EleutherAI/llemma_7b`</li><li>`togetherai:Gryphe/MythoMax-L2-13b`</li><li>`togetherai:Meta-Llama/Llama-Guard-7b`</li><li>`togetherai:Nexusflow/NexusRaven-V2-13B`</li><li>`togetherai:NousResearch/Nous-Capybara-7B-V1p9`</li><li>`togetherai:NousResearch/Nous-Hermes-2-Yi-34B`</li><li>`togetherai:NousResearch/Nous-Hermes-Llama2-13b`</li><li>`togetherai:NousResearch/Nous-Hermes-Llama2-70b`</li></ul> |

Aliases and custom commands:

| Name | Target |
|------|--------|
| `gpt2` | `huggingface_hub:gpt2` |
| `gpt3` | `openai:davinci-002` |
| `chatgpt` | `openai-chat:gpt-3.5-turbo` |
| `gpt4` | `openai-chat:gpt-4` |
| `ernie-bot` | `qianfan:ERNIE-Bot` |
| `ernie-bot-4` | `qianfan:ERNIE-Bot-4` |
| `titan` | `bedrock:amazon.titan-tg1-large` |
| `openrouter-claude` | `openrouter:anthropic/claude-3.5-sonnet:beta` |


## Example Prompt
Bellow is a prompt that may create a workable code

In [13]:
%%ai gpt4 -f code
write a python function to search jobs from usajobs,
retrive the maximal results per page and maximal pages,
extract the key informaiton in the search result,
store each job in a seperate mongodb document,
a mongodb database and collection is already set up,
do not set up the mongodb client and collection, use the collection  direclty.
user proivdes the agent, authorization key, job location, job keywords,  collection 


Cannot determine model provider from model ID `gpt4`.

To see a list of models you can use, run `%ai list`

If you were trying to run a command, run `%ai help` to see a list of commands.

In [14]:
import requests

def search_usajobs(agent, auth_key, location, keywords, collection):
    base_url = "https://data.usajobs.gov/api/search"
    headers = {
        "User-Agent": agent, 
        "Authorization-Key": auth_key,
        "Host": "data.usajobs.gov"
    }
    params = {
        "LocationName": location,
        "Keyword": keywords,
        "ResultsPerPage": 500
    }

    response = requests.get(base_url, headers=headers, params=params)
    data = response.json()
    num_pages = int(data['SearchResult']['UserArea']['NumberOfPages'])

    for page in range(1, num_pages + 1):
        params["Page"] = page
        response = requests.get(base_url, headers=headers, params=params)
        data = response.json()
        jobs = data['SearchResult']['SearchResultItems']

        for job in jobs:
            collection.insert_one(job)

In [15]:
search_usajobs(agent = email,
               auth_key = usa_jobs_key, 
               location = "Fairfax, VA",
               keywords= "AI",
               collection= job_collection)




## Example Code
Bellow is a code generated by AI that works

In [17]:
import requests

def search_jobs(agent, auth_key, job_location, job_keywords, collection):
    base_url = 'https://data.usajobs.gov/api/search'
    headers = {'User-Agent': agent, 'Authorization-Key': auth_key}
    params = {'LocationName': job_location, 'Keyword': job_keywords, 'ResultsPerPage': 500}

    page = 1
    while page <= 10:
        params['Page'] = page
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code != 200:
            break

        job_data = response.json()
        for job in job_data['SearchResult']['SearchResultItems']:
            job_info = job['MatchedObjectDescriptor']
            collection.insert_one(job_info)

        page += 1

Use the AI-generated code to collect `AI-related` jobs in `Fairfax, VA`. We also pass the `job_collection`, `api_key`, and `email` to the function.

In [18]:
search_jobs(collection= job_collection,
            auth_key=usa_jobs_key, 
            agent= email, 
            job_keywords= 'intelligence analysis',
            job_location= 'VA')

Display the number of collected jobs:

In [19]:
job_collection.estimated_document_count()

208